<a href="https://colab.research.google.com/github/juancamiloespana/RL_DS/blob/master/RL_SD_ventas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Se tiene por objetivo optimizar los parámetros de un modelo de dinámica de sistemas que modela la venta de un único producto. Los parámetros de interes y sus respectivos ragngos de posibles valores son:

* `initial_sales_force = (25,75)`
* `average_salary = (20000,30000)`
* `widget_price = (98,102)`
* `exit_rate = (0.15,0.25)`

La principal variable de salida es la utilidad total, calculada como la suma de las utilidades en los distntos periodos de tiempo. Sin embargo, hay otras variables que permiten describir el desempeño total del sistema al final de la simulación. Siendo así las variables de resultado son:

* `total_revenues`
* `size_of_sales_force`
* `total_hires`
* `total_departures`
* `total_widget_sales`



# Modelo de aprendizaje reforzado 


El modelo formulado considera los siguientes elementos:
* un modelo de dinámica de sistema `DSmodel` que con base en los parámetros de entrada calcula las variables de resultado basado en una simulación de un horizonte de tiempo `horizon` y un delta de tiempo `dt`.
* Un entorno (`environment`) del que en cada instante del tiempo se puede obtener una observación (`obs`) de su estado actual. 
* Una observación corresponde a un vector de 9 posiciones que contiene el valor actual de los cuatro parámetros de interes y el valor actual (resultado de correr el modelo de simulación con esos parámetros) para las cinco variables de resultado. 
> `[ 0 1 0 1 0 17 20]`
* Un agente (`agent`) que con base en una observación del estado actual del sistema toma una acción `a` respecto a los parámetros del módelo. El tamaño del espacio de acciones es el doble del número de acciones dado que se considera la posibilidad de incrementar el valor del parámetro (multiplicarlo por 1+δ) o disminuir el valor del parámetro, multiplicarlo por 1-δ). Así por ejemplo, las dos acciones posible para el parámetro `exit_rate` y un $\delta = 0.01$ se representan como las tuplas: $(exit\_rate, 0.99)$ y $(exit\_rate, 1.01)$
* La decisión del agente se basa en la predicción realizada por una red neuronal (`net`). Esta red recibe una observación y devuelve la probabilidad de cada una de las acciones



Implementaremos cada uno de los elementos mencionados, para ello primero instalamos los paquetes requeridos

In [ ]:
from collections import namedtuple
from collections import OrderedDict
import numpy as np
import itertools  
import plotly.graph_objects as go
import random
import copy


import torch
import torch.nn as nn
import torch.optim as optim

## Modelo de dinámica de sistemas

Esta clase implementa el modelo de dinámica de sistemas. Inicializa todos los parámetros y resultados en cero. Se implementa un metodo de inicialización de parámetros `initialise()` que permite definir valores diferentes para los parámetros. El método `run()` ejecuta el modelo y devuleve un diccionario con el valor de las cinco variables de resultado. 


In [ ]:
class DSmodel:
  def __init__(self, horizon, dt):
    self.horizon = horizon 
    self.dt = dt
    self.total_time = int(horizon/dt) 
    self.initial_sales_force = 0 
    self.average_salary = 0 
    self.widget_price = 0 
    self.exit_rate = 0 
    # Definición del nivel
    self.size_of_sales_force = 0
    # Definición de los flujos
    self.New_hires = 0
    self.Departures = 0
    # Definición de las variables auxiliares
    self.budgeted_size = 0
    self.sales_dep = 0
    self.annual_revenues = 0
    self.widget_sales = 0
    self.effectiveness_widgets = 0
    
    self.total_revenues = 0
    self.total_hires = 0
    self.total_departures = 0
    self.total_widget_sales = 0

    self.dict_results = OrderedDict()
    self.dict_results['total_revenues'] = 0
    self.dict_results['size_of_sales_force'] = 0
    self.dict_results['total_hires'] =  0
    self.dict_results['total_departures'] =  0
    self.dict_results['total_widget_sales'] =  0
                    

  def initialise(self,
                 initial_sales_force, 
                 average_salary,
                 widget_price,
                 exit_rate
                 ):
    self.initial_sales_force = initial_sales_force 
    self.average_salary = average_salary 
    self.widget_price = widget_price 
    self.exit_rate = exit_rate
    # Definición del nivel
    self.size_of_sales_force = 0
    # Definición de los flujos
    self.New_hires = 0
    self.Departures = 0
    # Definición de las variables auxiliares
    self.budgeted_size = 0
    self.sales_dep = 0
    self.annual_revenues = 0
    self.widget_sales = 0
    self.effectiveness_widgets = 0 

    self.total_revenues = 0
    self.total_hires = 0
    self.total_departures = 0
    self.total_widget_sales = 0

    self.dict_results = OrderedDict()
    self.dict_results['total_revenues'] = 0
    self.dict_results['size_of_sales_force'] = 0
    self.dict_results['total_hires'] =  0
    self.dict_results['total_departures'] =  0
    self.dict_results['total_widget_sales'] =  0

  def run(self):
    for i in range(self.total_time):
        if i == 0:
            self.size_of_sales_force = self.initial_sales_force
        else:
            self.size_of_sales_force += (self.New_hires - self.Departures) * self.dt
        
        # Caclula effectiveness_widgets
        if 0 <= self.size_of_sales_force < 600:
            self.effectiveness_widgets = 2
        if 600 <= self.size_of_sales_force < 800:
            self.effectiveness_widgets = 2+((1.8-2)*(self.size_of_sales_force-600)/(800-600))
        if 800 <= self.size_of_sales_force < 1000:
            self.effectiveness_widgets = 1.8+((1.6-1.8)*(self.size_of_sales_force-800)/(1000-800))
        if 1000 <= self.size_of_sales_force < 1200:
            self.effectiveness_widgets = 1.6+((0.8-1.6)*(self.size_of_sales_force-1000)/(1200-1000))
        else:
            self.effectiveness_widgets = 0.8+((0.4-0.8)*(self.size_of_sales_force-1200)/(1400-1200))

        self.widget_sales = self.size_of_sales_force * self.effectiveness_widgets * 365
        self.annual_revenues = self.widget_sales * self.widget_price / 1000000        
        self.sales_dep = self.annual_revenues * 0.5
        self.budgeted_size = self.sales_dep * 1000000 / self.average_salary
        self.New_hires = self.budgeted_size - self.size_of_sales_force
        self.Departures = self.size_of_sales_force * self.exit_rate    

        self.total_revenues += self.annual_revenues
        self.total_hires += self.New_hires
        self.total_departures += self.Departures
        self.total_widget_sales += self.widget_sales 

    self.dict_results['total_revenues'] = self.total_revenues
    self.dict_results['size_of_sales_force'] = self.size_of_sales_force
    self.dict_results['total_hires'] =  self.total_hires
    self.dict_results['total_departures'] =  self.total_departures
    self.dict_results['total_widget_sales'] =  self.total_widget_sales

    return self.dict_results
    


A continuación se crea un modelo y se ejecuta con datos de prueba para los parámetros



In [ ]:
model = DSmodel(20, 0.1)
model.initialise(initial_sales_force = 30,
                average_salary = 25000,
                widget_price = 100,
                exit_rate = 0.20)
results = model.run()
results

## Red neuronal para predecir acción 

La red considera una entrada de observaciones, una función `reLU` y un número `hidden_size` especificado de capas ocultas. La salida debe ser psoteriormente pasada a través de una función `Softmax()` para convertirla en las probabilidades de cada acción

In [ ]:
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x):
        return self.net(x)

Probemos la red con una observación fictica

## Clase `environment`

In [ ]:
class environment:
  '''    
    Attributes:
    

  '''
  def __init__(self, 
               list_param,
               list_results,
               parm_ranges,
               ds_model, 
               n_iter = 1, 
               delta = 0.01 ):     
    self.list_param = list_param # list of parameters' names
    self.list_results = list_results # list of result variables' names
    self.dict_max_results = {key:0 for key in self.list_results}
    self.parm_ranges = parm_ranges # dictionary with the feasible range of each parameter
    self.ds_model = ds_model # Dinamic system model
    self.dict_obs = OrderedDict() # dictionary with the observation variables \
                                  # (parameters + result variables)      
    self.iter = n_iter # number of parameters changes in the episode 
    self.moves = 0 # counter of changes
    self.iter_max_revenue = 0 # iteration in which the maximum revenue takes place
    self.delta = delta # change in the parameter value
    # creates the action space
    #f1 = lambda x: (x, 1-self.delta)
    #f2 = lambda x: (x, 1+self.delta)  
    #self.action_space = [f(x) for x in list(self.parm_ranges.keys()) for f in (f1,f2)]
    self.action_space = list(itertools.product(list(self.parm_ranges.keys()), \
                               [1-self.delta, 1+self.delta]))
    # initialize parameters in a random value within the feasible ranges
    self.initialise_param()
    # intialize result variables to 0
    for result in self.list_results:        
      self.dict_obs[result] = 0
  

  def initialise_param(self):
    ''' initialize parameters in a random value within the feasible ranges'''
    for param in self.list_param:
      self.dict_obs[param] = round(random.uniform(self.parm_ranges[param][0], \
                                                   self.parm_ranges[param][1]), 2)
    
 
  def step(self, a):
    """ Takes an action and updates the environment. """
    is_done = False
    action = self.action_space[a]
    param_bu = self.dict_obs[action[0]] # back up of the param value
    param_new = self.dict_obs[action[0]] * action[1] # new value of the parameter
    if param_new < self.parm_ranges[action[0]][0] or param_new > self.parm_ranges[action[0]][1]:
      # if the action is not feasible look rendmly for a fesible action 
      while True:
        action_rnd = random.choices(self.action_space)[0]
        param_test = self.dict_obs[action_rnd[0]] * action_rnd[1]
        if param_test >= self.parm_ranges[action_rnd[0]][0] and param_test <= self.parm_ranges[action_rnd[0]][1]:
          action = action_rnd
          break
    self.dict_obs[action[0]] *= action[1] #update the value of the parameter
    # initialise and run the DS model
    self.ds_model.initialise(self.dict_obs['initial_sales_force'],
                                self.dict_obs['average_salary'],
                                self.dict_obs['widget_price'],
                                self.dict_obs['exit_rate'])    
    results = self.ds_model.run()
    # Update observation
    for key, value in results.items():
      self.dict_obs[key] = value

    # update maximum results 
    for key in self.list_results:
      if self.dict_obs[key] > self.dict_max_results[key]:
        self.dict_max_results[key] = self.dict_obs[key]
        # keep track of the iteration in which the best revenue was reached 
        if key == 'total_revenues':
          self.iter_max_revenue = self.moves

    self.moves += 1 # update the moves counter 
    # check if the maximun number of moves it reached 
    if self.moves >= self.iter:
      is_done = True
    return is_done

  def getObs(self):
    ''' Gets the values of the dictionary of observation '''
    return list(self.dict_obs.values())

  def getScaledObs(self):
    obs_dict = copy.deepcopy(self.dict_obs)
    for key in list(self.parm_ranges.keys()):
      obs_dict[key] = (obs_dict[key] - self.parm_ranges[key][0])/ \
                 (self.parm_ranges[key][1]-self.parm_ranges[key][0])   
    for key in  self.list_results:
      if self.dict_max_results[key] != 0:
        obs_dict[key] = obs_dict[key]/self.dict_max_results[key]

    return list(obs_dict.values())

  def reset(self):
    """ Resets results while keeping settings"""
    self.dict_obs = OrderedDict()
    self.initialise_param()
    for result in self.list_results:        
      self.dict_obs[result] = 0
    self.dict_max_results = {key:0 for key in self.list_results}
    
    self.moves = 0
    self.max_revenue = 0
    self.iter_max_revenue = 0

## Clase `agent`

In [ ]:
class agent:
  '''    
    Attributes:
    sm (Softmax): softmax operator 
    eps (real [0,1]): controls diversity

  '''
  def __init__(self, sm, eps):    
    self.action = (-99, None) # Define null action
    self.sm = sm    
    self.eps = eps


  def choose_action(self, env, net):
    '''Obtains and observation from the environment and chooses and 
    action based on the probabilities given for a neural network'''    
    p = np.random.rand() # Generate random number
    if p < self.eps:
      # Randomly select an action
      self.action = np.random.choice(len(env.action_space))
    else:
      # Take greedy action chosing the one with highest probability
      obs = env.getObs()
      #print(obs)
      obs_v = torch.FloatTensor([obs])
      act_probs_v = self.sm(net(obs_v))
      act_probs = act_probs_v.data.numpy()[0]      
      self.action = np.random.choice(len(act_probs), p=act_probs)  

    
    return self.action



## Iteración en batches

In [ ]:
# usa namedtuples para guardar los pasos de cada episodio y los episodios 
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
all_revenues = []


def iterate_batches(env, agent, net, batch_size):
  '''Generate batches of the parameter tunning solutions. 
     Each solution consist of steps = [observation, actions] taken to solve the 
     problem. The steps represents the iterative modification of the parameters'
     values. The batch collects several solutions (batch_size) '''  
  batch = []
  episode_reward = 0.0
  episode_steps = []  
  sm = nn.Softmax(dim=1)
  env.reset()

  # Repeat until enough solutions are built
  while True:
    a = agent.choose_action(env, net)     
    is_done = env.step(a)
    #obs = env.getObs()
    obs = env.getScaledObs()
    step = EpisodeStep(observation=obs, action=a)
    episode_steps.append(step) 
    all_revenues.append(env.dict_obs['total_revenues'])
    # if a solutions is complete, it reset the environment to start a new solution   
    if is_done:
      # keeeps the steps only until the iteration in which 
      # the max revvene was reached
      episode_steps = episode_steps[0:env.iter_max_revenue+1]
      e = Episode(reward=env.dict_max_results['total_revenues'], steps=episode_steps)
      batch.append(e)
      env.reset()
      episode_steps = []
      #all_revenues.append(env.dict_obs['total_revenues'])
      # If enough solutions are generated it returns the batch
      if len(batch) == batch_size:        
        yield batch
        batch = []

## Selecciona las mejores soluciones 

Escoge las mejores soluciones de un batch y extrae las observaciones y la acción que el agente tomó en ellas

In [ ]:
def filter_batch(batch, percentile):
    '''Selects the best solutions given a percentile  ''' 
    rewards = list(map(lambda s: s.reward, batch))
    reward_bound = np.percentile(rewards, percentile)
    position_best = np.argmax(rewards)
    reward_max = rewards[position_best]    

    train_obs = []
    train_act = []
    for reward, steps in batch:
        if reward < reward_bound:
            continue
        train_obs.extend(map(lambda step: step.observation, steps))
        train_act.extend(map(lambda step: step.action, steps))
    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)
    
    return train_obs_v, train_act_v, reward_bound, reward_max

# Experimentación

## Búsqueda aleatoria

Una primera forma de resolver el problema sería mediante búsqueda aletoria. Es decir, corriendo muchas veces el modelo de dinámica de sistema con valores aletarorios de los parámetros. 

In [ ]:
param_dict = {}
param_dict['initial_sales_force'] = [-99, ((10,100))]
param_dict['average_salary'] = [-99, (10000,50000)]
param_dict['widget_price'] = [-99, (50,100)]
param_dict['exit_rate'] = [-99, (0.1,0.5)]
horizon = 20
dt = 0.1
incunbent_hist =[]
incunbent = 0
iter_hist = []
repetitions = 20000 # number of repetitions

# Creates the moel
ds_model = DSmodel(horizon, dt)

def initialise_parameters(param_dict):
  ''' initialize parameters randomly '''
  for param in list(param_dict.keys()):
      param_dict[param][0] = round(random.uniform(param_dict[param][1][0], \
                                               param_dict[param][1][1]), 2)
      

for i in range(repetitions):
  initialise_parameters(param_dict)
  ds_model.initialise(param_dict['initial_sales_force'][0],
                      param_dict['average_salary'][0],
                      param_dict['widget_price'][0],
                      param_dict['exit_rate'][0])
      
  results = ds_model.run()
  results
  if results['total_revenues'] > incunbent:
    incunbent = results['total_revenues']
  incunbent_hist.append(incunbent)
  iter_hist.append(results['total_revenues'])


# Print graph
fig = go.Figure()
x = list(range(repetitions))
fig.add_trace(go.Scatter(x = x, y = incunbent_hist, name= "incumbent"))
fig.add_trace(go.Scatter(x = x, y = iter_hist, name= "best_episode")) 
fig.show()

## Aprendizaje reforzado

In [ ]:
# Parameters
HIDDEN_SIZE = 128
BATCH_SIZE = 20
PERCENTILE = 70
N_EPOCH = 10
EPSILON = 0.01
n_iter = 100 #iterations within each episode
horizon = 20
dt = 0.1
delta = 0.05 # percentage in which the parameter is changed

parm_ranges = OrderedDict()
parm_ranges['initial_sales_force'] = (10,100)
parm_ranges['average_salary'] = (10000,50000)
parm_ranges['widget_price'] = (50,100)
parm_ranges['exit_rate'] = (0.1,0.5)

list_param = list(parm_ranges.keys())
list_results = ['total_revenues', 'size_of_sales_force',\
                    'total_hires', 'total_departures', 'total_widget_sales']
observacion_list = list_param + list_results 

# Lists to save the observed solutions
incunbent_hist =[0]
incunbent = 0
best_list = [0]
all_revenues = []

# Create neural network 
net = Net(len(observacion_list), HIDDEN_SIZE, 2*len(list_param))
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.01)
sm = nn.Softmax(dim=1)
# Create  agent and environment
ds_model = DSmodel(horizon, dt)
# Create  agent and environment
env = environment(list_param, list_results, parm_ranges, ds_model, n_iter, delta)
#print(env.action_space)
ag = agent(sm, EPSILON)



# Iterate over batches. Each time a batch is created the neural network is
# updated
for epoch, batch in enumerate(iterate_batches(env, ag, net, BATCH_SIZE)):
  
  print('epoch_no', epoch, len(all_revenues))  
  # Filters the batch
  obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE)
  
  # Updates the incumbent (best solution)
  if reward_m > incunbent:
    incunbent = reward_m
  incunbent_hist.append(incunbent)
  best_list.append(reward_m)

  # Update network
  optimizer.zero_grad()
  action_scores_v = net(obs_v)
  loss_v = objective(action_scores_v, acts_v)
  loss_v.backward()
  optimizer.step()

  # stops if the number of episodes is reached
  if epoch >= N_EPOCH:
    break


Imprimamos la forma como evluciona la solución con respecto a la mejor solución de cada una de las epocas o batches

In [ ]:
fig = go.Figure()
x = list(range(N_EPOCH))
fig.add_trace(go.Scatter(x = x, y = incunbent_hist, name= "incumbent"))
fig.add_trace(go.Scatter(x = x, y = best_list, name= "best_episode"))
 
fig.show()

veamos ahora el detalle de cada una de las soluciones evaluadas por el algoritmo


In [ ]:
fig = go.Figure()
x = list(range(len(all_revenues)))
fig.add_trace(go.Scatter(x = x, y = all_revenues, name= "revenue"))
 
fig.show()